In [39]:
import helpers as h
import json
import numpy as np
import os
import pandas as pd
import sqlite3
import sys

from tqdm import tqdm

Load population figures for European countries. Data source: https://worldpopulationreview.com/continents/europe/cities

The aim of the project is to compare the standard of living in European cities. Therefore, due to the fact that Russia's territory within Europe is small-Russian cities were excluded from further analysis.

In [43]:
population_df = pd.read_csv('table_data.csv')

selected_country_df = (population_df
    .sort_values(
        by=['country', 'population'], 
        ascending=[True,False]
    ))
    
selected_country_df = selected_country_df.loc[selected_country_df['country']!='Russia']
selected_country_df.to_csv(os.path.join('data','selected_countries.csv'))

A list of cities and countries with data was downloaded from numbeo.com. The list is extracted based on the population table data from the previous step.

In [41]:
numbeo_url = 'https://www.numbeo.com/cost-of-living'
numbeo = h.get_content_url(page_url=numbeo_url)

europe_country = selected_country_df.country.unique()

country_city = []
for country in tqdm(europe_country):

    country_url = f'{numbeo_url}/country_result.jsp?country={country}'
    country_soup = h.get_content_url(page_url=country_url)

    city_select = country_soup.find("select",{"name":"city"}) 
    city_list = city_select.find_all('option')
    for city in city_list:
        if city.text != '--- Select city---':
            country_city.append((country,city.text.strip()))
country_city_df = pd.DataFrame(country_city, columns=('country', 'city'))
country_city_df.head(5)

100%|██████████| 36/36 [00:31<00:00,  1.14it/s]


,country,city
0,Albania,Apollonia
1,Albania,Ballsh
2,Albania,Berat
3,Albania,Burrel
4,Albania,Dhermi


In [44]:
country_city_df.to_csv(os.path.join('data','countries_cities.csv'))

Downloading data from numbeo.com and then saving it as a data dictionary {state: {city: data}}'. During data acquisition, a problem was noticed regarding the local currency, and for this reason it was decided to download the data in Euros.

In [ ]:
country_city_dict = {}

for _, country, city in tqdm(list(country_city_df.itertuples())):
    city_url = f'{numbeo_url}/in/{city}'
    city_soup = h.get_content_url(city_url)
    city_dict = {}
    if 'These cities seems to have a name similar' in city_soup.find('p').text:
        for a in city_soup.select('p a', href=True): 
            if a.text and country in a.text: 
                city_url_eur = a['href'] + '?displayCurrency=EUR'
    else:
        city_url_eur = city_url + '?displayCurrency=EUR'

    city_soup = h.get_content_url(city_url_eur)

    if country in country_city_dict:
        city_dict = country_city_dict[country]
    else:
        city_dict = {}
        country_city_dict[country] = city_dict

    city_dict[city] = h.get_data_from_table(
        city_soup,
        converters={
            'Restaurants': str,
            'Edit': str,
            'Range': str
        }
    ).rename(
        columns={
            'Restaurants': 'Type',
            'Edit': 'Avg_price',
            'Range': 'Range_price'
        }
    )

Due to the long time of data retrieval, saved the obtained result in a .json file. This way we keep a backup of the data.

In [ ]:
city_json = {}

for country in country_city_dict:
    city_json[country] ={}
    for city in country_city_dict[country]:
        city_json[country][city] = json.loads(country_city_dict[country][city].to_json())

with open(os.path.join('data','numbeo.json'), 'w') as f:
    json.dump(city_json, f)